## Практическое задание к уроку 5 по теме "Part-of-Speech разметка, NER, извлечение отношений".

Задание 1. Написать теггер на данных с русским языком
1. проверить UnigramTagger, BigramTagger, TrigramTagger и их комбинации
2. написать свой теггер как на занятии, попробовать разные
векторайзеры, добавить знание не только букв но и слов  
3. сравнить все реализованные методы, сделать выводы  
  

Загрузим библиотеки и датасеты:

In [1]:
import nltk
from nltk.tag import UnigramTagger, BigramTagger, TrigramTagger, DefaultTagger
import pandas as pd
import pyconll
from sklearn.feature_extraction.text import HashingVectorizer, TfidfVectorizer, CountVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

In [2]:
path = '../../Теория/Lesson_5/'

In [3]:
full_train = pyconll.load_from_file(path + 'dataset_ru/ru_syntagrus-ud-train-a.conllu')
full_train_b = pyconll.load_from_file(path + 'dataset_ru/ru_syntagrus-ud-train-b.conllu')
full_train_c = pyconll.load_from_file(path + 'dataset_ru/ru_syntagrus-ud-train-c.conllu')

full_train.extend([*full_train_b, *full_train_c])

full_test = pyconll.load_from_file(path + 'dataset_ru/ru_syntagrus-ud-dev.conllu')

In [4]:
fdata_train = []
for sent in full_train:
    fdata_train.append([(token.form, token.upos) for token in sent])
    
fdata_test = []
for sent in full_test:
    fdata_test.append([(token.form, token.upos) for token in sent])

Протестируем различные тэггеры из библиотеки nltk и их комбинации.  
Внесём получившиеся значения accuracy в таблицу:

In [5]:
%%time

default_tagger = DefaultTagger('NOUN')
possible_options = {
    'Trigram': 'TrigramTagger(fdata_train)',
    'Bigram': 'BigramTagger(fdata_train)',
    'Unigram': 'UnigramTagger(fdata_train)',
    'Default': 'default_tagger',
    'Trigram + Default': 'TrigramTagger(fdata_train, backoff=default_tagger)',
    'Trigram + Bigram': 'TrigramTagger(fdata_train, backoff=BigramTagger(fdata_train))',
    'Trigram + Bigram + Default': 'TrigramTagger(fdata_train, backoff=BigramTagger(fdata_train, backoff=default_tagger))',
    'Trigram + Unigram': 'TrigramTagger(fdata_train, backoff=UnigramTagger(fdata_train))',
    'Trigram + Unigram + Default': 'TrigramTagger(fdata_train, backoff=UnigramTagger(fdata_train, backoff=default_tagger))',
    'Trigram + Bigram + Unigram': 'TrigramTagger(fdata_train, backoff=BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train)))',
    'Trigram + Bigram + Unigram + Default': 'TrigramTagger(fdata_train, backoff=BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train, backoff=default_tagger)))',
    'Bigram + Default': 'BigramTagger(fdata_train, backoff=default_tagger)',
    'Bigram + Unigram': 'BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train))',
    'Bigram + Unigram + Default': 'BigramTagger(fdata_train, backoff=UnigramTagger(fdata_train, backoff=default_tagger))',
    'Unigram + Default': 'UnigramTagger(fdata_train, backoff=default_tagger)'    
}

scores = []

for name, tagger in possible_options.items():
    scores.append((round(eval(tagger).accuracy(fdata_test), 3)))

CPU times: user 1min 11s, sys: 341 ms, total: 1min 11s
Wall time: 1min 11s


In [6]:
comparison = pd.DataFrame(scores, index=possible_options.keys(), columns=['Score']).sort_values('Score', ascending=False)
comparison

,Score
Trigram + Unigram + Default,0.912
Trigram + Bigram + Unigram + Default,0.912
Bigram + Unigram + Default,0.912
Unigram + Default,0.906
Bigram + Unigram,0.884
Trigram + Unigram,0.883
Trigram + Bigram + Unigram,0.883
Unigram,0.878
Trigram + Bigram + Default,0.861
Bigram + Default,0.861


Лучшим среди одиночных тэггеров оказался Unigram, поэтому все  
лучшие комбинации были с ним в составе. Добавление в качестве  
последнего тэггера дефолтный даёт прирост всем комбинациям, что  
довольно логично, ведь мы вместо None на незнакомых словах ставим  
тэг существительного, и иногда попадаем.

Напишем свой тэггер. Попробую сделать тэггер, который на  
тренировочном датасете будет запоминать, какой тэг стоял  
до и после текущего слова. Тэг текущего слова будет выбираться  
равным либо тэгу предыдущего, либо тэгу следующего, смотря каких  
тэгов было больше на трейне. В данном случае тэг текущего слова  
никак не будет учитываться, и, скорее всего, такой тэггер покажет  
плохой результат, но мы потренируемся в написании.

In [7]:
from collections import defaultdict

In [8]:
class MyTagger(nltk.tag.SequentialBackoffTagger):
    def __init__(self, train, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Словари, где ключами являются текущие слова, а значениями - словари, 
        # где ключи - тэги предыдущего (следующего) слова, а значения - количество
        # этих тэгов:
        self.prev_values_dict = {} 
        self.next_values_dict = {} 
                                   
        for sent in train:
            words = [pair[0] for pair in sent]
            tags = [pair[1] for pair in sent]
            for i, word in enumerate(words):
                if word is None:
                    continue
                word = word.lower()
                
                # Если текущего слова нет в словаре, то создаём  для него defaultdict:
                if i != 0:
                    if word not in self.prev_values_dict:
                        self.prev_values_dict[word] = defaultdict(int)
                    self.prev_values_dict[word][tags[i-1]] += 1
                    
                if i != len(words) - 1:
                    if word not in self.next_values_dict:
                        self.next_values_dict[word] = defaultdict(int)
                    self.next_values_dict[word][tags[i+1]] += 1
    
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word is None:
            return None
        word = word.lower()
        next_count = 0
        prev_count = 0
        
        # Находим наиболее популярные тэги предыдущего и следующего слова для текущего слова
        if word in self.prev_values_dict:
            prev_popular, prev_count = sorted(self.prev_values_dict[word].items(), key=lambda x: x[1], reverse=True)[0]
        if word in self.next_values_dict:
            next_popular, next_count = sorted(self.next_values_dict[word].items(), key=lambda x: x[1], reverse=True)[0]
        
        # Если слова нет ни в одном словаре, то возвращаем None
        if (prev_count == 0) and (next_count == 0):
            return None
        
        # Возвращаем наиболее популярный тэг из тэгов следующего (предыдущего) слова
        if prev_count >= next_count:
            return prev_popular
        elif prev_count < next_count:
            return next_popular

Обучим тэггер и проверим его точность:

In [9]:
my_tagger = MyTagger(fdata_train)

In [10]:
my_tagger.accuracy(fdata_test)

0.06064847971873169

Как и ожидалось, тэггер оказался плох, намного хуже даже дефолтного.  
Теперь напишем более простой тэггер, который является аналогом Unigram:

In [11]:
class MyUniTagger(nltk.tag.SequentialBackoffTagger):
    def __init__(self, train, *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        # Снова словарь словарей, но теперь в ключах defaultdict  
        # будут тэги текущего слова
        self.dictionary = {}
        
        for sent in train:
            for word, tag in sent:
                if word is None:
                    continue
                    
                word = word.lower()
                if word not in self.dictionary:
                    self.dictionary[word] = defaultdict(int)
                self.dictionary[word][tag] += 1
            
    
    def choose_tag(self, tokens, index, history):
        word = tokens[index]
        if word is None:
            return None
        word = word.lower()
        
        # Возвращаем наиболее популярный тэг текущего слова
        if word in self.dictionary:
            return sorted(self.dictionary[word].items(), key=lambda x: x[1], reverse=True)[0][0]
        
        return None

Оценим этот тэггер:

In [12]:
my_unitagger = MyUniTagger(fdata_train)

In [13]:
my_unitagger.accuracy(fdata_test)

0.8829611302819194

Добавим новые данные в таблицу метрик:

In [14]:
comparison_mytaggers = pd.concat((comparison, pd.DataFrame(round(my_tagger.accuracy(fdata_test), 3), 
                                                       index=['MyTagger'], columns=['Score'])), axis=0)
comparison_mytaggers = pd.concat((comparison_mytaggers, pd.DataFrame(round(my_unitagger.accuracy(fdata_test), 3), 
                                                       index=['MyUniTagger'], columns=['Score'])), axis=0)
comparison_mytaggers = pd.concat((comparison_mytaggers, pd.DataFrame(round(MyTagger(fdata_train, backoff=default_tagger)\
                                                             .accuracy(fdata_test), 3), 
                                                       index=['MyTagger + Default'], columns=['Score'])), axis=0)
comparison_mytaggers = pd.concat((comparison_mytaggers, pd.DataFrame(round(MyUniTagger(fdata_train, backoff=default_tagger)\
                                                             .accuracy(fdata_test), 3), 
                                                       index=['MyUniTagger + Default'], columns=['Score'])), axis=0)

comparison_mytaggers.sort_values('Score', ascending=False, inplace=True)
comparison_mytaggers

,Score
Trigram + Unigram + Default,0.912
Trigram + Bigram + Unigram + Default,0.912
Bigram + Unigram + Default,0.912
MyUniTagger + Default,0.908
Unigram + Default,0.906
Bigram + Unigram,0.884
Trigram + Unigram,0.883
Trigram + Bigram + Unigram,0.883
MyUniTagger,0.883
Unigram,0.878


Наш тэггер оказался чуть лучше Unigram тэггера nltk, а его комбинация  
с дефолтным показала второй результат.

Теперь обучим логистическую регрессию на мультиклассовую  
классификацию, чтобы предсказывать тэги. Подготовку данных  
будем производить с помощью встроенных в sklearn векторайзеров.  
Попробуем варианты с векторизацией символов и их комбинаций, а затем  
слов и их комбинаций.  
Сделаем предобработку:

In [15]:
train_tok = []
y_train = []

# В трейн не будем записывать пропущенные значения токенов или тэгов
for sent in fdata_train:
    for word, tag in sent:
        if (word is None) or (tag is None):
            continue
        train_tok.append(word)
        y_train.append(tag)
        
test_tok = []
y_test = []

# В тест будем записывать всё для более честного сравнения с тэггерами
for sent in fdata_test:
    for word, tag in sent:
        if word is None:
            test_tok.append('NO_WORD')
        else:
            test_tok.append(word)
        if tag is None:
            y_test.append('NO_TAG')
        else:
            y_test.append(tag)

Посмотрим на распределение классов:

In [16]:
pd.Series(y_test).value_counts(normalize=True)

NOUN      0.235940
PUNCT     0.190025
VERB      0.111400
ADJ       0.098333
ADP       0.089309
ADV       0.050674
PRON      0.048467
CCONJ     0.036929
PROPN     0.035634
PART      0.033368
DET       0.027769
SCONJ     0.018654
NUM       0.011290
AUX       0.009050
NO_TAG    0.001725
X         0.000872
SYM       0.000404
INTJ      0.000156
dtype: float64

Может быть, в данном случае метрика accuracy и не лучший выбор, но,  
во-первых, на такой метрике мы проверяли тэггеры. Во-вторых, самый  
популярный класс имеет долю 23%, а это далеко от метрики лучших тэггеров.  
Так что случайная модель всё равно не покажет хороший результат.

Сначала применим векторизацию по символам:

In [17]:
vectorizer_char = TfidfVectorizer(ngram_range=(1, 6), analyzer='char')

In [18]:
X_train = vectorizer_char.fit_transform(train_tok)

In [19]:
X_test = vectorizer_char.transform(test_tok)

In [20]:
X_train.shape, X_test.shape

((1204640, 304810), (153590, 304810))

In [21]:
%%time
lr_char = LogisticRegression(random_state=29, solver='sag')
lr_char.fit(X_train, y_train)

CPU times: user 48.7 s, sys: 180 ms, total: 48.9 s
Wall time: 48.9 s


LogisticRegression(random_state=29, solver='sag')

In [22]:
pred = lr_char.predict(X_test)

In [23]:
lr_char_score = accuracy_score(y_test, pred)
lr_char_score

0.944885734748356

Неплохо! Помимо Tfidf были опробованы HashingVectorizer и  
CountVectorizer. Результат Hashing оказался хуже, чем Tfidf.  
Результат CountVectorizer получился выше примерно на 0,5%, но  
при этом модель обучалась в несколько раз дольше. Для занятия  
первой строчки в таблице хватило и этого варианта.

Применим векторизацию по словам:

In [24]:
vectorizer_word = TfidfVectorizer(ngram_range=(1, 2), analyzer='word')

In [25]:
X_train = vectorizer_word.fit_transform(train_tok)

In [26]:
X_test = vectorizer_word.transform(test_tok)

In [27]:
%%time
lr_word = LogisticRegression(random_state=29, solver='sag')
lr_word.fit(X_train, y_train)

CPU times: user 22.6 s, sys: 152 ms, total: 22.7 s
Wall time: 22.7 s


LogisticRegression(random_state=29, solver='sag')

In [28]:
pred = lr_word.predict(X_test)

In [29]:
lr_word_score = accuracy_score(y_test, pred)
lr_word_score

0.7619636695097337

Здесь Tfidf и CountVectorizer получили примерно одинаковый результат.  
HashingVectorizer хуже примерно на 0,5%.  
Внесём все данные в таблицу:

In [30]:
comparison_mytaggers_vectorizers = pd.concat((comparison_mytaggers, 
                                              pd.DataFrame(round(lr_char_score, 3), index=['Char Tfidf + Logistic Regression'], 
                                                                                    columns=['Score'])), axis=0)
comparison_mytaggers_vectorizers = pd.concat((comparison_mytaggers_vectorizers, 
                                              pd.DataFrame(round(lr_word_score, 3), index=['Word Tfidf + Logistic Regression'], 
                                                                                    columns=['Score'])), axis=0)
comparison_mytaggers_vectorizers.sort_values('Score', ascending=False, inplace=True)
comparison_mytaggers_vectorizers

,Score
Char Tfidf + Logistic Regression,0.945
Trigram + Unigram + Default,0.912
Bigram + Unigram + Default,0.912
Trigram + Bigram + Unigram + Default,0.912
MyUniTagger + Default,0.908
Unigram + Default,0.906
Bigram + Unigram,0.884
Trigram + Unigram,0.883
Trigram + Bigram + Unigram,0.883
MyUniTagger,0.883


С большим отрывом лидирует логистическая регрессия с Tfidf векторизацией  
по символам. Векторизация по словам оказалась значительно хуже, и это объяснимо:  
ведь о части речи может говорить какая-то часть слова, например, окончание, а не  
всё слово целиком.